In [1]:
import os
import numpy as np
import keras
from keras.datasets import reuters, imdb
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN, GRU, Dense, Dropout, Activation, Embedding, TimeDistributed, Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import pandas as pd

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
EMBEDDING_DIM = 50
# load in training/test set
data = pd.read_csv('tweets.160k.random.csv', encoding='utf-8')
data.head()

,label,id,date,query,user,text
0,4,1985770747,Sun May 31 17:44:25 PDT 2009,NO_QUERY,vozabala,Getting ready for another week of fun and game...
1,0,2322735567,Wed Jun 24 23:10:08 PDT 2009,NO_QUERY,liannecab,"http://twitpic.com/8cp6u - I want it, sooo bad"
2,0,1972997427,Sat May 30 10:16:49 PDT 2009,NO_QUERY,nadhirarchangel,iloveyousincethe1stgradeitsthefirsttimewemet ...
3,0,2230992481,Thu Jun 18 17:53:46 PDT 2009,NO_QUERY,doughamlin,@extendr I can add :skype links but :aim links...
4,4,2053227537,Sat Jun 06 03:46:32 PDT 2009,NO_QUERY,Mariallama,just woke up at to rain. . . on the plus side ...


In [74]:
import gensim
import wget
def read_input(input_file):
    print("reading file")
    result = []
    with open(input_file,'r') as f:
        lines = f.readlines()
        for line in range(0,4000):
#             print(line)
            ## this does all the preprocessing
            lines[line] = lines[line].strip().strip(',').strip("'")
            lines[line] = lines[line].rstrip()
            lines[line] = lines[line].lower()
            gensim.utils.simple_preprocess(lines[line])
            for char in lines[line]:
                result.append(char)
    return result
                        
data_file = 'shakespeare.txt'
documents = list(read_input(data_file))
print('done reading')

reading file
done reading


In [75]:
print(documents[:100])

['1', '6', '0', '9', 't', 'h', 'e', ' ', 's', 'o', 'n', 'n', 'e', 't', 's', 'b', 'y', ' ', 'w', 'i', 'l', 'l', 'i', 'a', 'm', ' ', 's', 'h', 'a', 'k', 'e', 's', 'p', 'e', 'a', 'r', 'e', '1', 'f', 'r', 'o', 'm', ' ', 'f', 'a', 'i', 'r', 'e', 's', 't', ' ', 'c', 'r', 'e', 'a', 't', 'u', 'r', 'e', 's', ' ', 'w', 'e', ' ', 'd', 'e', 's', 'i', 'r', 'e', ' ', 'i', 'n', 'c', 'r', 'e', 'a', 's', 'e', 't', 'h', 'a', 't', ' ', 't', 'h', 'e', 'r', 'e', 'b', 'y', ' ', 'b', 'e', 'a', 'u', 't', 'y', "'", 's']


In [76]:
d = dict([(y,x+1) for x,y in enumerate(sorted(set(documents)))])

In [88]:
print(d)
print(len(documents))

{' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '<': 22, '>': 23, '?': 24, '[': 25, ']': 26, 'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 'k': 37, 'l': 38, 'm': 39, 'n': 40, 'o': 41, 'p': 42, 'q': 43, 'r': 44, 's': 45, 't': 46, 'u': 47, 'v': 48, 'w': 49, 'x': 50, 'y': 51, 'z': 52}
146600


In [139]:
def prepare_dataset(documents,mapping):
    dataset = []
    chars  = []
    y_values = []
    temp = []
    for char in range(len(documents)):
        if char%51==0 and char!= 0:
            y_values.append(mapping[documents[char]])
            temp.append(chars)
            temp.append(y_values)
            dataset.append(temp)
            chars  = []
            y_values = []
            temp = []
            continue
        chars.append(mapping[documents[char]])
    
    return dataset
        
        
            
        

In [140]:
prepare_dataset(documents,d)

[[[11,
   16,
   10,
   19,
   46,
   34,
   31,
   1,
   45,
   41,
   40,
   40,
   31,
   46,
   45,
   28,
   51,
   1,
   49,
   35,
   38,
   38,
   35,
   27,
   39,
   1,
   45,
   34,
   27,
   37,
   31,
   45,
   42,
   31,
   27,
   44,
   31,
   11,
   32,
   44,
   41,
   39,
   1,
   32,
   27,
   35,
   44,
   31,
   45,
   46,
   1],
  [29]],
 [[44,
   31,
   27,
   46,
   47,
   44,
   31,
   45,
   1,
   49,
   31,
   1,
   30,
   31,
   45,
   35,
   44,
   31,
   1,
   35,
   40,
   29,
   44,
   31,
   27,
   45,
   31,
   46,
   34,
   27,
   46,
   1,
   46,
   34,
   31,
   44,
   31,
   28,
   51,
   1,
   28,
   31,
   27,
   47,
   46,
   51,
   4,
   45,
   1,
   44],
  [41]],
 [[45,
   31,
   1,
   39,
   35,
   33,
   34,
   46,
   1,
   40,
   31,
   48,
   31,
   44,
   1,
   30,
   35,
   31,
   28,
   47,
   46,
   1,
   27,
   45,
   1,
   46,
   34,
   31,
   1,
   44,
   35,
   42,
   31,
   44,
   1,
   45,
   34,
   41,
   47,
   38,
   30,
   1,

In [141]:
import numpy as np

numpy = np.array(prepare_dataset(documents,d))

In [142]:
numpy.shape

(2874, 2)

In [144]:
print(len(numpy[5][0]))

50


In [163]:
x_data = []
for tup in numpy:
    if len(tup[0])!=50:
        print('wrong')
        continue
    x_data.append(tup[0])
    
# print(x_data)
print(np.array(x_data).shape)

wrong
(2873, 50)


In [164]:
x_data = np.expand_dims(np.array(x_data),2)
print(x_data.shape)


(2873, 50, 1)


In [176]:
y_data = []
for tup in numpy:
    if len(tup[0])!=50:
        print('wrong')
        continue
    y_data.append(tup[1])

y_data = keras.utils.to_categorical(y_data)
print(y_data)

wrong
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [177]:
model = Sequential()
# model.add(Embedding(len(word_index)+1,EMBEDDING_DIM, weights=[embedding_matrix], trainable=False))
model.add(LSTM(256,input_shape=(x_data.shape[1],x_data.shape[2])))
# model.add(LSTM(128))
model.add(Dropout(0.5))
model.add((Dense(y_data.shape[1],activation='softmax')))
# model.add(Activation('softmax'))

In [178]:
model.compile(loss='binary_crossentropy',optimizer='adam')
model.fit(x_data,y_data,epochs=5,batch_size=32)

Epoch 1/5
2873/2873 [==============================] - 16s 6ms/step - loss: 0.0797
Epoch 2/5
2873/2873 [==============================] - 15s 5ms/step - loss: 0.0758
Epoch 3/5
2873/2873 [==============================] - 15s 5ms/step - loss: 0.0747
Epoch 4/5
2873/2873 [==============================] - 15s 5ms/step - loss: 0.0740
Epoch 5/5
2873/2873 [==============================] - 15s 5ms/step - loss: 0.0736


In [205]:
string = 'out of grief and impatience. answer\'d neglectingly'
pred = []
for char in string:
    pred.append(d[char])

# x_pred = np.expand_dims(pred,1)
pred = np.reshape(pred, (1, 50, 1))

# x_pred = np.expand_dims(pred,1)
print(pred.shape)
print(pred)

(1, 50, 1)
[[[41]
  [47]
  [46]
  [ 1]
  [41]
  [32]
  [ 1]
  [33]
  [44]
  [35]
  [31]
  [32]
  [ 1]
  [27]
  [40]
  [30]
  [ 1]
  [35]
  [39]
  [42]
  [27]
  [46]
  [35]
  [31]
  [40]
  [29]
  [31]
  [ 9]
  [ 1]
  [27]
  [40]
  [45]
  [49]
  [31]
  [44]
  [ 4]
  [30]
  [ 1]
  [40]
  [31]
  [33]
  [38]
  [31]
  [29]
  [46]
  [35]
  [40]
  [33]
  [38]
  [51]]]


In [208]:
outp = model.predict(pred)
print(len(outp[0]))

52
